# Stock-Duration-To-Come-Back-From-Crisis

Generating a table with the worst crisis for a stock over a given period of time. And the time it took for the stock to come back to the initial price it had fallen from.

This will give an historical view of the kind of crises a company faced in its history, and the duration it took the company to recover form that crisis.


| Company   | Crisis     |  Low | from High | Change | got back by | after (years)
|-----------|----------- |------|-----------|--------|-------------|--------------
| Boeing    | 1973-03-10 | \$10 |  \$20     | -50%   |1976-09-10   |3.5

In [9]:
import pandas as pd
import yfinance as yf # Module to retrieve data on financial instruments (similar to 'yahoo finance')

## Parameters

### `period`

Period of time under study. 

This is the period of time during which we want to find the crisis for the stock under study.

For example, it can be the last 5 years, the last 10 years or the maximum period for which we have data.

In [11]:
period = 'max' # valid period values: 1y,2y,5y,10y,ytd,max

### `time_window`

The time window is the moving time interval used to calculate the loss of the sotck.
At each point in time, the loss will be calculated compared to a reference point dated at the beginning of that time window.

This way, the loss calculated will be representative of the loss over that rolling time window.

E.g.: 
`time_window = '20'`
The change will be calculated over a rolling time window of 20 trading days.


In [27]:
time_window = 20 # in open trading days

### `large_loss`

A stock will be considered to have a crisis if the loss it suffers over the `time_window` is larger than `large_loss`.

In [13]:
large_loss = 30 # large loss, in %

### `stock`

Captures the sock (company) to be studied. 

The variable should be set with the official "Ticker" of the company.

E.g: `stock = 'AAPL'` for Ticker AAPL (Ticker of the company Apple).

In [12]:
stock = 'BA' # stock under study. Boeing in this case.

## Implementation

### Get the stock historical price data for the period under study

Use the yfinance module to download the daily data for the period. And store the result in a Pandas DataFrame.

In [21]:
df = yf.download(tickers=stock, period= period, interval='1d') # Download the data

df.insert(0, 'Company', stock) # Insert a column "Company" with the stock name in it

df.head() # Display the first rows of the data

[*********************100%***********************]  1 of 1 completed


,Company,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
1962-01-02,BA,0.145748,0.823045,0.837449,0.823045,0.837449,352200
1962-01-03,BA,0.148663,0.839506,0.851852,0.835391,0.835391,710400
1962-01-04,BA,0.147205,0.831276,0.853909,0.831276,0.839506,911000
1962-01-05,BA,0.144290,0.814815,0.835391,0.792181,0.831276,880400
1962-01-08,BA,0.144654,0.816872,0.829218,0.804527,0.814815,473600


### Calculathe Change over the `time_window` rolling period

Create a column to store the reference price at the beginning of the time window.

In [30]:
df['Reference Price'] = df.Close.shift(time_window)

In [29]:
df

,Company,Adj Close,Close,High,Low,Open,Volume,Reference Price
Date,,,,,,,,
1962-01-02,BA,0.145748,0.823045,0.837449,0.823045,0.837449,352200,NaN
1962-01-03,BA,0.148663,0.839506,0.851852,0.835391,0.835391,710400,NaN
1962-01-04,BA,0.147205,0.831276,0.853909,0.831276,0.839506,911000,NaN
1962-01-05,BA,0.144290,0.814815,0.835391,0.792181,0.831276,880400,NaN
1962-01-08,BA,0.144654,0.816872,0.829218,0.804527,0.814815,473600,NaN
1962-01-09,BA,0.145019,0.818930,0.837449,0.810700,0.816872,576800,NaN
1962-01-10,BA,0.145019,0.818930,0.833333,0.816872,0.818930,352200,NaN
1962-01-11,BA,0.146476,0.827160,0.845679,0.820988,0.820988,692400,NaN
1962-01-12,BA,0.150485,0.849794,0.855967,0.835391,0.835391,972000,NaN


At the beginning of the data, we cannot get a reference point back the entire time window because we do not have the data before that point.

We will replace the missing values by the oldest possible price point available: the first historical price point available in the data.

In [34]:
df.fillna(value=df.iloc[0]['Close'],inplace=True)

In [35]:
df

,Company,Adj Close,Close,High,Low,Open,Volume,Reference Price
Date,,,,,,,,
1962-01-02,BA,0.145748,0.823045,0.837449,0.823045,0.837449,352200,0.823045
1962-01-03,BA,0.148663,0.839506,0.851852,0.835391,0.835391,710400,0.823045
1962-01-04,BA,0.147205,0.831276,0.853909,0.831276,0.839506,911000,0.823045
1962-01-05,BA,0.144290,0.814815,0.835391,0.792181,0.831276,880400,0.823045
1962-01-08,BA,0.144654,0.816872,0.829218,0.804527,0.814815,473600,0.823045
1962-01-09,BA,0.145019,0.818930,0.837449,0.810700,0.816872,576800,0.823045
1962-01-10,BA,0.145019,0.818930,0.833333,0.816872,0.818930,352200,0.823045
1962-01-11,BA,0.146476,0.827160,0.845679,0.820988,0.820988,692400,0.823045
1962-01-12,BA,0.150485,0.849794,0.855967,0.835391,0.835391,972000,0.823045
